In [53]:
# get data
import pandas as pd
from sqlalchemy import create_engine, inspect

# Create an SQLite engine
engine = create_engine('sqlite:///data/ESNQ_DB.db')

table_names = inspect(engine).get_table_names()
print(table_names)
ticker = 'ES1D'
df = pd.read_sql(ticker, engine)
# df = df[-500:]
print(df.info())

['ES1D', 'ES1M', 'ES30M', 'ES4H', 'ES5M', 'NQ1D', 'NQ1M', 'NQ30M', 'NQ4H', 'NQ5M']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6835 entries, 0 to 6834
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     6835 non-null   int64  
 1   datetime  6835 non-null   object 
 2   symbol    6835 non-null   object 
 3   open      6835 non-null   float64
 4   high      6835 non-null   float64
 5   low       6835 non-null   float64
 6   close     6835 non-null   float64
 7   volume    6835 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 427.3+ KB
None


In [54]:
# clean df
# backtesting ValueError: `data` must be a pandas.DataFrame with columns 'Open', 'High', 'Low', 'Close', and (optionally) 'Volume'
df.columns = df.columns.str.capitalize()
df['Date'] = pd.to_datetime(df['Datetime'])
# df.set_index('Date', inplace=True)
df.drop(['Index','Symbol','Volume'], axis=1, inplace=True)

df.tail()

,Datetime,Open,High,Low,Close,Date
6830,2024-09-08 17:00:00,5410.00,5493.0,5405.25,5479.50,2024-09-08 17:00:00
6831,2024-09-09 17:00:00,5488.25,5506.0,5448.25,5504.00,2024-09-09 17:00:00
6832,2024-09-10 17:00:00,5499.25,5567.5,5412.00,5561.25,2024-09-10 17:00:00
6833,2024-09-11 17:00:00,5558.00,5607.0,5540.25,5602.25,2024-09-11 17:00:00
6834,2024-09-12 17:00:00,5598.50,5641.5,5597.25,5629.75,2024-09-12 17:00:00


In [55]:
import numpy as np
df['Signal'] = np.random.randint(-1,2,len(df))
df

,Datetime,Open,High,Low,Close,Date,Signal
0,1997-09-08 17:00:00,933.75,941.25,932.75,934.00,1997-09-08 17:00:00,0
1,1997-09-09 17:00:00,933.75,934.25,914.50,915.25,1997-09-09 17:00:00,-1
2,1997-09-10 17:00:00,916.00,917.50,900.50,908.50,1997-09-10 17:00:00,0
3,1997-09-11 17:00:00,908.00,926.25,904.00,924.00,1997-09-11 17:00:00,0
4,1997-09-14 17:00:00,933.50,939.75,929.00,931.50,1997-09-14 17:00:00,1
...,...,...,...,...,...,...,...
6830,2024-09-08 17:00:00,5410.00,5493.00,5405.25,5479.50,2024-09-08 17:00:00,-1
6831,2024-09-09 17:00:00,5488.25,5506.00,5448.25,5504.00,2024-09-09 17:00:00,-1
6832,2024-09-10 17:00:00,5499.25,5567.50,5412.00,5561.25,2024-09-10 17:00:00,1
6833,2024-09-11 17:00:00,5558.00,5607.00,5540.25,5602.25,2024-09-11 17:00:00,-1


In [56]:
from backtesting import Strategy

class SignalStrategy(Strategy):
    def init(self):
        pass

    def next(self):
        current_signal = self.data.Signal[-1]
        if current_signal == 1:
            if not self.position:
                self.buy()
        elif current_signal == -1:
            if self.position:
                self.position.close()

In [57]:
from backtesting import Backtest

# bt =Backtest(df, SignalStrategy, cash=1000000, commission=.002, exclusive_orders=True)
# bt = Backtest(df, SignalStrategy, cash=100000, commission=0.002)
bt = Backtest(df, SignalStrategy, cash=100000)

stats = bt.run()
print(stats)

/var/folders/1v/lpjl6vj906s43r87d8bhvdb80000gn/T/ipykernel_77942/2787161326.py:5: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, SignalStrategy, cash=100000)


Start                                     0.0
End                                    6834.0
Duration                               6834.0
Exposure Time [%]                   66.891002
Equity Final [$]                     205892.0
Equity Peak [$]                      233239.5
Return [%]                            105.892
Buy & Hold Return [%]              502.756959
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -44.482128
Avg. Drawdown [%]                   -2.694831
Max. Drawdown Duration                 3001.0
Avg. Drawdown Duration              68.112245
# Trades                               1152.0
Win Rate [%]                        54.774306
Best Trade [%]                      12.583862
Worst Trade [%]                     -12.90631
Avg. Trade [%]                    

In [58]:
bt.plot()

GridPlot(id='p2823', ...)